In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [2]:
# investigator 
# ablilities with embedid values not capturing correctly
# Complex deck reqs not being captured correctly
# some must includes not being captured correctly

In [3]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [4]:
##########################################Get soup request#########################################################
def get_soup(url):
    '''Child of get_skills_df
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page and return
    return soup.find(id='list')

In [5]:
def clean_ability_string(ability):
    
    # replace icon html with matching word in all caps
    icon_types = [
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        ability = ability.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        ability = ability.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    ability = ability.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    ability = ability.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    ability = ability.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>',]
    
    for item in dirt:
        
        ability = ability.replace(item,'')

    return ability

In [6]:
def get_stat_line(results):
    
    stat_line = results.find('div', class_='card-props').text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1], stat_list[2], stat_list[3]


def get_stam_line(results):
    
    stat_line = results.find('div', class_='card-props').find_next().text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1]


def get_deck_reqs(results, faction):

    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    pattern = r'(?:\b(?:Deck Size|Deckbuilding Options|Deckbuilding Requirements|Additional Setup):|\b\s*)([^:\n]+)'

    deck_split = re.findall(pattern, str(deck))

    deck_size = re.search('[0-9]{2}', deck_split[0]).group()

    deck_options = deck_split[1].replace('Deckbuilding Requirements (do not count toward deck size)', '').replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = deck_split[2].replace('.Additional Setup', '')

    try:

        setup = deck_split[3]

    except:

        setup = NONE

    return deck_size, deck_options, must_include, setup

In [7]:
def get_invest_traits(results):
    '''Scrapes and returns investigator card data'''

    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

    willpower, intellect, combat, agility = get_stat_line(results)

    health, sanity = get_stam_line(results)
   
    ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

    try:
        
        flavor = results.find_all('div', class_='card-flavor small')[1].text.replace('\n', '').replace('\t', '')

    except:
        
        flavor = ''
        
    deck_size, deck_options, must_include, setup = get_deck_reqs(results, faction)
    
    return [title, 
            faction, 
            tipe, 
            traits, 
            willpower, 
            intellect, 
            combat, 
            agility, 
            health, 
            sanity,
            ability, 
            artist,
            expansion,
            flavor,
            deck_size,
            deck_options,
            must_include,
            setup]

In [8]:
# make empty df

def get_invest_df():

    investigator_traits = { 'title':[],
                            'faction':[],
                            'type':[],
                            'traits':[],                      
                            'willpower':[],
                            'intellect':[],
                            'combat':[],
                            'agility':[],                       
                            'health':[],
                            'sanity':[],                     
                            'ability':[],
                            'artist':[],
                            'expansion':[],
                            'flavor':[],
                            'deck_size':[],
                            'deck_options':[],
                            'must_include':[],
                            'setup':[]}

    # for each url get player card info from that page and add each element to skill_traits
    for url in invest:

        # make html request to arkham db and parse using BS
        results = get_soup(url)

        # get list of card elements card elements
        trait_list = get_invest_traits(results)

        print(f'Getting investigator card {trait_list[0]}...')

        # itterate through card element titles and add each to a dictionary
        for i, key in enumerate(investigator_traits):

            investigator_traits[key].append(trait_list[i])

    print("Making dataframe...")

    return pd.DataFrame(investigator_traits)

In [9]:
get_invest_df()

NameError: name 'NONE' is not defined

In [ ]:
# [title, test_icons, traits, willpower, intellect, combat, agility, health, sanity, faction, ability, tipe, flavor, artist, expansion]

In [ ]:
# Investigator Traits

results = get_soup('https://arkhamdb.com/card/06003')

title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

willpower, intellect, combat, agility = get_stat_line(results)

health, sanity = get_stam_line(results)

ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

flavor = results.find_all('div', class_='card-flavor small')[1].text.replace('\n', '').replace('\t', '')

deck_size, deck_options, must_include = get_deck_reqs(results)

In [ ]:
results = get_soup('https://arkhamdb.com/card/06003')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text.replace('\n', '').replace('\t', '')

In [ ]:
results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text



In [ ]:
def get_deck_reqs(results, faction):
    
    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    deck_split = re.split(':', deck)

    deck_size = re.search('[0-9]{2}', deck_split[1]).group()
    
    deck_options = re.search('^(.*?\.)', deck_split[2]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = re.search('^(.*?\.)', deck_split[3]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')
    
    return deck_size, deck_options, must_include

In [ ]:


def get_deck_reqs(results, faction):

    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    pattern = r'(?:\b(?:Deck Size|Deckbuilding Options|Deckbuilding Requirements|Additional Setup):|\b\s*)([^:\n]+)'

    deck_split = re.findall(pattern, str(deck))

    deck_size = re.search('[0-9]{2}', deck_split[0]).group()

    deck_options = deck_split[1].replace('Deckbuilding Requirements (do not count toward deck size)', '').replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = deck_split[2].replace('.Additional Setup', '')

    try:

        setup = deck_split[3]

    except:

        setup = NONE

    return deck_size, deck_options, must_include, setup

In [20]:
results = get_soup(invest[26])

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

deck = results.find('div', class_=f'card-text border-{faction.lower()}').text

pattern = r'Deck Size'

deck_split = re.findall(pattern, deck)

deck

'\n When an investigator at your location would reveal any number of chaos tokens: Reveal 2 additional tokens. Of the revealed tokens, choose and cancel 2 non- tokens, or 1  token. (Limit once per round.) effect: +1. If this effect is canceled or ignored, draw 1 card.\n'

In [15]:
results.find('div', class_=f'card-text border-{faction.lower()}').text

'\n When an investigator at your location would reveal any number of chaos tokens: Reveal 2 additional tokens. Of the revealed tokens, choose and cancel 2 non- tokens, or 1  token. (Limit once per round.) effect: +1. If this effect is canceled or ignored, draw 1 card.\n'

In [ ]:
# Input string
input_string = '''Deck Size: 30 Secondary Class Choice: At deck creation, choose Guardian (), Seeker (), 
                or Survivor (). Deckbuilding Options: Rogue cards () level 0-5, Neutral cards level 0-5, 
                up to 10 level 0-1 events and/or skills of your chosen secondary class. Deckbuilding Requirements 
                (do not count toward deck size): Bounty Contracts, 2 copies of Tony's .38 Long Colt, Tony's Quarry,
                1 random basic weakness.'''




In [ ]:
title

In [ ]:
faction

In [ ]:
tipe

In [ ]:
traits

In [ ]:
willpower

In [ ]:
intellect

In [ ]:
combat

In [ ]:
agility

In [ ]:
health

In [ ]:
sanity

In [ ]:
ability

In [ ]:
artist

In [ ]:
expansion

In [ ]:
deck_size

In [ ]:
flavor

In [ ]:
deck_size

In [ ]:
deck_options

In [ ]:
must_include 